In [1]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *
conf = SparkConf().setMaster("local").setAppName("PySpark_feature_eng")
spark = SparkSession.builder.getOrCreate()
print(spark)

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

In [3]:
print(spark.version)

3.0.0-preview2


In [ ]:
import pandas as pd
import os
# Download data into a pandas dataframe 


# Download data into a pandas dataframe 
review_df = pd.read_csv(r"C:\Users\raaga\Desktop\cmpe272\Medicare_Provider_Util_Payment_PUF_CY2017.txt", error_bad_lines=False)

In [2]:
import pandas as pd 
import dask.dataframe as dd
import os
from tqdm import tqdm

TRAIN_PATH = r'C:\Users\raaga\Desktop\cmpe272\Medicare_Provider_Util_Payment_PUF_CY2017.txt'

In [3]:
%%time
# Assume we only know that the csv file is somehow large, but not the exact size
# we want to know the exact number of rows

# Method 1, using file.readlines. Takes about 20 seconds.
with open(TRAIN_PATH) as file:
    n_rows = len(file.readlines())

print (f'Exact number of rows: {n_rows}')

Exact number of rows: 9847445
Wall time: 22.4 s


In [6]:
df_tmp = pd.read_csv(TRAIN_PATH, sep='\t',nrows=5)
df_tmp.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt
0,1,CPT copyright 2016 American Medical Associatio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,99217.0,Hospital observation care discharge,N,100.0,96.0,100.0,73.398800,325.780000,56.827200,57.492400
2,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,99218.0,Hospital observation care typically 30 minutes,N,26.0,25.0,26.0,100.080000,449.000000,78.460000,79.306154
3,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,99219.0,Hospital observation care typically 50 minutes,N,52.0,51.0,52.0,136.380000,614.000000,102.807692,103.895385
4,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,99220.0,Hospital observation care typically 70 minutes...,N,59.0,59.0,59.0,190.363729,755.932203,141.293559,142.865763


In [7]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   npi                               5 non-null      int64  
 1   nppes_provider_last_org_name      5 non-null      object 
 2   nppes_provider_first_name         4 non-null      object 
 3   nppes_provider_mi                 0 non-null      float64
 4   nppes_credentials                 4 non-null      object 
 5   nppes_provider_gender             4 non-null      object 
 6   nppes_entity_code                 4 non-null      object 
 7   nppes_provider_street1            4 non-null      object 
 8   nppes_provider_street2            0 non-null      float64
 9   nppes_provider_city               4 non-null      object 
 10  nppes_provider_zip                4 non-null      float64
 11  nppes_provider_state              4 non-null      object 
 12  nppes_provid

In [9]:
traintypes = {'nppes_provider_street2  ': 'float32',
              'nppes_provider_last_org_name': 'str',
              'nppes_provider_first_name':'str',
              'nppes_credentials':'str',
              'nppes_entity_code':'str',
              'hcpcs_description':'str',
              'hcpcs_code':'float32',
              ' bene_day_srvc_cnt':'float32',
               'average_Medicare_allowed_amt' : 'float32',
              'average_submitted_chrg_amt' : 'float32',
              'average_Medicare_payment_amt' : 'float32',
              'average_Medicare_standard_amt' : 'float32',              
              }

cols = list(traintypes.keys())

In [11]:
chunksize = 3_000_000

In [ ]:
%%time
df_list = [] # list to hold the batch dataframe

for df_chunk in tqdm(pd.read_csv(TRAIN_PATH, sep ='\t', usecols=cols, dtype=traintypes, chunksize=chunksize)):
     
    # Neat trick from https://www.kaggle.com/btyuhas/bayesian-optimization-with-xgboost
    # Using parse_dates would be much slower!
    df_chunk['pickup_datetime'] = df_chunk['pickup_datetime'].str.slice(0, 16)
    df_chunk['pickup_datetime'] = pd.to_datetime(df_chunk['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
    
    # Can process each chunk of dataframe here
    # clean_data(), feature_engineer(),fit()
    
    # Alternatively, append the chunk to list and merge all
    df_list.append(df_chunk) 